In [1]:
import io, os, uuid, yaml
import numpy as np
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from src.pipeline import utils, azure_tools


Se cargan las credenciales

In [2]:
credentials = utils.load_credentials("blob_storage")

Código de prueba para la Azure Function

In [145]:
# blob  = BlobClient.from_connection_string(credentials['conn_string'], container_name = "reports-ingredion", blob_name="test Brazil.xlsm")
# with open("./test Brazil P&L.xlsm", "rb") as file:
#     blob.upload_blob(file)

In [6]:
# container_client = azure_tools.get_container_client("reports-ingredion",credentials['conn_string'])
# blob_list = azure_tools.get_blob_list(container_client)
# container_client.delete_blob("test Brazil.xlsm")

Se obtienen la lista de archivos en *Raw*

In [7]:
container_input = "reports-ingredion"
container_client = azure_tools.get_container_client(container_input,credentials['conn_string'])

In [8]:
# name of the existing container
blob_list = azure_tools.get_blob_list(container_client)

In [9]:
len(blob_list)

120

Se obtiene lista de archivos en *clean*

In [26]:
container_output = "intermediate-ingredion"

In [27]:
container_client = azure_tools.get_container_client(container_output,credentials['conn_string'])
blob_list = azure_tools.get_blob_list(container_client)

In [28]:
len(blob_list)

8

In [13]:
blob_list

['test Brazil.xlsm']

In [ ]:
# blob_client = BlobServiceClient.from_connection_string(credentials['conn_string'])
# container_client = blob_client.create_container(container_output)

In [6]:
df = azure_tools.transform_pl_files(blob_list[:6], container_input, container_output, credentials['conn_string'])

Processing:  1-2016 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2016 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2016.parquet
Processing:  1-2017 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2017 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2017.parquet
Processing:  1-2018 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2018 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2018.parquet


## Read data

In [17]:
container_client = azure_tools.get_container_client(container_output,credentials['conn_string'])
blob_list = azure_tools.get_blob_list(container_client)
#container_client.delete_blob(file)

In [18]:
len(blob_list)

0

In [19]:
for file in blob_list:
    container_client.delete_blob(file)

In [151]:
df_test = azure_tools.read_transform_pl(container_output,credentials['conn_string'])

ValueError: No objects to concatenate

In [24]:
df_test.shape

(375011, 48)

In [25]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375011 entries, 0 to 7328
Data columns (total 48 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Fiscal year/period                 375011 non-null  float64
 1   Company Code                       375011 non-null  object 
 2   Unnamed: 8                         375011 non-null  float64
 3   Ship to party                      375011 non-null  object 
 4   Unnamed: 10                        375011 non-null  object 
 5   Material                           370077 non-null  object 
 6   Commercial Name                    375011 non-null  object 
 7   Sales Qty Total MT                 373705 non-null  float64
 8   3P Sales Qty Total MT              322723 non-null  float64
 9   Gross Revenue USD                  375011 non-null  float64
 10  Discounts USD                      375011 non-null  float64
 11  New Net Revenue                    375011

In [153]:
container_client = azure_tools.get_container_client(container_output,credentials['conn_string'])
stream = container_client.download_blob(blob_list[0])
file = io.BytesIO()
stream.readinto(file)
df_f = pd.read_parquet(file)

In [154]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2415 entries, 0 to 2414
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Fiscal year/period                 2415 non-null   float64
 1   Company Code                       2415 non-null   object 
 2   Unnamed: 8                         2415 non-null   float64
 3   Ship to party                      2415 non-null   object 
 4   Unnamed: 10                        2415 non-null   object 
 5   Material                           2415 non-null   object 
 6   Commercial Name                    2415 non-null   object 
 7   Sales Qty Total MT                 2415 non-null   float64
 8   3P Sales Qty Total MT              2285 non-null   float64
 9   Gross Revenue USD                  2415 non-null   float64
 10  Discounts USD                      2415 non-null   int64  
 11  New Net Revenue                    2415 non-null   float